In [2]:
import json
import os
import stat
import time
import datetime

from pyspark import Row, SparkConf
from pyspark.sql import SparkSession
#,functions
# from pyspark import SparkConf
# conf = SparkConf()
# conf.set("spark.master","local[*]") \
#     .set("spark.app.name","spark-advance") \
#     .set("spark.storage.memoryFraction", "0.5")

In [ ]:
spark = SparkSession \
    .builder.appName("spark-advance") \
    .master("local[*]") \
    .getOrCreate()
print("spark session built.")

In [23]:
# load json from relative path
df = spark.read\
    .format("json")\
    .option("pathGlobFilter","*.json")\
    .load("../DataLake/searches/")
print(df.count())

65973


In [ ]:
#save parquet in specific path
df.write\
    .format("parquet")\
    .mode("overwrite")\
    .save("../parquet/searches")
print("save parquet succeed.")

In [19]:
#sql parquet in specific path XXXXXXXXXXXXXXXXX
df = spark.sql('select * from parquet."../parquet/searches"')
print(df.count())
#we cant specify relative path in spark.sql XXXXXXXXXXXXX

ParseException: 
no viable alternative at input 'parquet."./searches"'(line 1, pos 22)

== SQL ==
select * from parquet."./searches"
----------------------^^^


In [24]:
#save in hive
df.write \
    .mode("overwrite") \
    .saveAsTable("searches")
print("save hive succeed.")

save hive succeed.


In [20]:
#save hive in specific path
df.write \
    .mode("overwrite") \
    .option("path","../hive/") \
    .saveAsTable("searches")
print("save hive in specific path succeed.")

save hive succeed.


In [25]:
#query hive specific
df = spark.sql("select * from searches")
print(df.count())

65973
save hive succeed.


In [87]:
#create table config succeed.
spark.sql("create table if not exists parquet.config(key string,name string)")
print("create table config succeed.")

AnalysisException: Hive support is required to CREATE Hive TABLE (AS SELECT);
'CreateTable `parquet`.`config`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, Ignore


In [108]:
# find max_modified_date in files of path
fileStatsObj = os.stat( "../DataLake/searches/" )
modificationTime = time.ctime ( fileStatsObj [ stat.ST_MTIME ] )
print("Last Modified Time : ", modificationTime )

max_filename = ""
max_time =0
for root, dirs, files in os.walk('../DataLake/searches/'):
    max_filename = max(max_filename, max(str(os.stat(root+"/"+file).st_mtime)+"-"+file for file in files) )
    max_time = max(max_time, max(os.stat(root+"/"+file).st_mtime for file in files) )
print(max_filename)
print(max_time)
t = datetime.datetime.fromtimestamp(max_time)
print(type(t))
print(t.strftime('%Y-%m-%dT%H:%M:%S'))
# print(time.ctime(max(os.path.getmtime(file) for file in files)))
# print(time.ctime(max(os.stat(file).st_mtime for file in files)))
# print(max(str(os.stat(file).st_mtime)+"-"+file for file in files))

Last Modified Time :  Mon Feb 14 10:42:00 2022
1644856071.685726-searches-part-00004-20200126.json
1644856071.685726
<class 'datetime.datetime'>
2022-02-14T19:57:51


In [95]:
try:
    df=(spark.read.format("parquet").load("config"))
    print("exists")
    df.show()
except:
    print("not")
    rdd = [Row(key='last_modified_date', value='2021-05-25T19:58:04')]
    df = spark.createDataFrame(rdd)
    df.write.parquet("config")

exists
+------------------+-------------------+
|               key|              value|
+------------------+-------------------+
|last_modified_date|2021-05-25T19:58:04|
+------------------+-------------------+



In [103]:
# spark.sql("insert into parquet.config values ('searches.last_file_date','2021-05-25T19:58:04')")
# read last modified date
df = spark.sql("select value from parquet.config where key='last_modified_date'")
df.show()
print(df.collect()[0][0])

+-------------------+
|              value|
+-------------------+
|2021-05-25T19:58:04|
+-------------------+

2021-05-25T19:58:04


In [107]:
#spark.sql("insert into parquet.config values ('searches.last_file_date','2021-05-25T19:58:04')")

#read new ones
df = spark.read \
    .format("json") \
    .option("pathGlobFilter","*.json") \
    .load("../DataLake/searches/", modifiedAfter=datetime.datetime(2021,5,26))
print(df.count())
#     .load("../DataLake/searches/", modifiedAfter=datetime.datetime() "2021-05-25T19:58:04")
# 2021-05-25T19:58:04
# .option("modifiedAfter","2021-05-25T19:58:04")
# 2021-05-25T19:58:04
# 65,973


65973


In [3]:
# create sparksession on hive
spark = SparkSession\
    .builder.appName("test")\
    .master("local")\
    .config("spark.sql.warehouse.dir","../spark-warehouse",SparkConf())\
    .getOrCreate()
print("hive spark session built.")


hive spark session built.


In [114]:
# saveAsTable in hive storage
rdd = [Row(key='k1', value='value1')
    ,Row(key='k2', value='value2')
    ,Row(key='k3', value='value3')]
df = spark.createDataFrame(rdd)
df.write\
    .mode("overwrite")\
    .format("parquet")\
    .saveAsTable("test")
print("write test table succed")

write test table succed


In [4]:
# read table on hive
df = spark.table("test")
df.show()

AnalysisException: Table or view not found: test;
'UnresolvedRelation [test], [], false


In [5]:
# query table on hive
df = spark.sql("select * from test")
df.show()

AnalysisException: Table or view not found: test; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [test], [], false


In [4]:
# write config to file
import json
conf = {"k1":"value1"
    ,"k2":"value2"
    ,"k3":"value3"}
with open("config.json", "w") as configfile:
    json.dump(conf, configfile)
    configfile.close()

In [5]:
# read config from json file
with open("config.json", "r") as configfile:
    conf = json.load(configfile)
print(conf)

{'k1': 'value1', 'k2': 'value2', 'k3': 'value3'}
